In [ ]:
!pip install google_genai
!pip install cadquery jupyter-cadquery


In [ ]:
import traceback
import json
import math
from google import genai
from google.genai import types
from kaggle_secrets import UserSecretsClient
import cadquery as cq

user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

MODEL_ID = "models/gemini-2.0-flash-exp"

In [ ]:
# 1. The basic instruction/role for the AI
instruction = """You are an expert assistant helping generate Python code for 3D modeling using the CadQuery library.
Generate only the Python code based on the user's request. Make sure the final object is assigned to a variable named 'result'.
"""

# 2. The examples (Few-Shot) - User/Assistant pattern
examples = """
User: Create a 10x10x10 mm cube.
Assistant:
import cadquery as cq
result = cq.Workplane("XY").box(10, 10, 10)

User: Make a solid cylinder with radius 5 and height 20.
Assistant:
import cadquery as cq
result = cq.Workplane("XY").cylinder(20, 5)

User: Create a 20x20x5 block and cut a hole of radius 3 through the center of the top face.
Assistant:
import cadquery as cq
result = cq.Workplane("XY").box(20, 20, 5).faces(">Z").workplane().circle(3).cutThruAll()
"""

In [ ]:
user_request = "A plate 50mm long, 30mm wide, and 3mm thick with four 3mm diameter holes, one in each corner, 5mm inset from the edges"

final_prompt = f"""{instruction}

Here are some examples:
{examples}

Now, fulfill this request:
User: {user_request}
Assistant:
"""

In [ ]:
response = client.models.generate_content(model=MODEL_ID,contents=final_prompt)
generated_code_text = response.text

In [ ]:
def cleanCode(uncleaned_code_text):
    cleaned_code_text = uncleaned_code_text.strip()
    if cleaned_code_text.startswith("```python"):
        cleaned_code_text = cleaned_code_text[len("```python"):].strip()
    elif cleaned_code_text.startswith("```json"):
        cleaned_code_text = cleaned_code_text[len("```json"):].strip()
    elif cleaned_code_text.startswith("```"):
        cleaned_code_text = cleaned_code_text[len("```"):].strip()

    if cleaned_code_text.endswith("```"):
        cleaned_code_text = cleaned_code_text[:-len("```")].strip()

    #print(cleaned_code_text)
    return cleaned_code_text

generated_code_text = cleanCode(generated_code_text)

In [ ]:
def execute_and_display(generated_python_code): # Renamed parameter for clarity
    print("--- Attempting to Execute Code ---")
    print(generated_python_code)
    print("------------------------------------")
    execution_scope = {'cq': cq} # Pre-populate scope with cadquery module alias
    try:
        # Execute the generated code string
        # Pass globals() AND the local scope dict.
        # Pre-populating with 'cq' helps if the generated code forgets the import sometimes (though yours includes it)
        exec(generated_python_code, globals(), execution_scope)

        # Retrieve the 'result' object from the scope
        cadquery_object = execution_scope.get('result')

        # Check and display
        if cadquery_object and isinstance(cadquery_object, cq.Workplane): # More specific check
            print("\nCode executed successfully. Displaying model...")
            display(cadquery_object) # Use display() which works with jupyter-cadquery
        elif 'result' not in execution_scope:
             print("\nError: The generated code did not create a 'result' variable in the execution scope.")
             # ADD DEBUGGING: Print the scope to see what *was* created
             print("Execution scope contains keys:", list(execution_scope.keys())) # See what variables ARE defined
        else:
             # ADD DEBUGGING: 'result' exists but isn't a Workplane object
             print("\nError: 'result' variable exists but is not a valid CadQuery Workplane object.")
             print("Generated code might be incorrect or incomplete.")
             print("Type of 'result' variable:", type(cadquery_object))
             print("Value of 'result' variable:", cadquery_object)


    except Exception as e:
        print(f"\n--- Exception Occurred During Execution ---")
        print(f"Error Type: {type(e)}")
        print(f"Error Message: {e}")
        print("\nTraceback:")
        traceback.print_exc() # Print the full traceback to diagnose the error location
        print("\n--- Generated Code That Caused Error ---")
        print(generated_python_code) # Print the code that was exec'd
        print("---------------------------------------")

In [ ]:
execute_and_display(generated_code_text)